<a href="https://colab.research.google.com/github/saadz-khan/federated-learning/blob/master/PySyft_Basics_%26_Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Syft Basics

## Checking Versions

In [ ]:
import sys
import torch
print(torch.__version__)
print(sys.version)

1.4.0
3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


## Install & Importing Dependencies 

In [ ]:
!pip install syft==0.2.9

In [ ]:
import syft as sy
hook = sy.TorchHook(torch)

## Fun with Jake

Creating Jake

In [ ]:
jake = sy.VirtualWorker(hook, id="jake")
print("Jake has: " + str(jake._objects))

Sending Data to Jake

In [ ]:
x = torch.tensor([1, 2, 3, 4, 5])
x = x.send(jake)                          # x is a pointer to tensor sent to Jake
print("x: " + str(x))
print("Jake has: " + str(jake._objects))

Reading Jake's Data

In [ ]:
x = x.get()
print("x: " + str(x))
print("Jake has: " + str(jake._objects))

## Say Hi to John

Sending Data to John

In [ ]:
john = sy.VirtualWorker(hook, id="john")
x = x.send(jake)                          # x is a pointer to tensor sent to Jake
x = x.send(john)                          # John now has Pointer to Jake's Data
print("x: " + str(x))
print("John has: " + str(john._objects))
print("Jake has: " + str(jake._objects))

Cleaning Up Data

In [ ]:
jake.clear_objects()
john.clear_objects()
print("Jake has: " + str(jake._objects))
print("John has: " + str(john._objects))

Moving Data from Jake to John

In [ ]:
y = torch.tensor([6, 7, 8, 9, 10]).send(jake)
y = y.move(john)
print(y)
print("Jake has: " + str(jake._objects))
print("John has: " + str(john._objects))

## Additive Secret Sharing

Encryption

In [ ]:
import random

# setting Q to a very large prime number
Q = 23740629843760239486723


def encrypt(x, n_share=3):
    r"""Returns a tuple containg n_share number of shares
    obtained after encrypting the value x."""

    shares = list()
    for i in range(n_share - 1):
        shares.append(random.randint(0, Q))
    shares.append(Q - (sum(shares) % Q) + x)
    return tuple(shares)


print("Shares: " + str(encrypt(3)))

Decryption

In [ ]:
def decrypt(shares):
    r"""Returns a value obtained by decrypting the shares."""

    return sum(shares) % Q


print("Value after decrypting: " + str(decrypt(encrypt(3))))

Homomorphic Encryption

In [ ]:
def add(a, b):
    r"""Returns a value obtained by adding the shares a and b."""

    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)


x, y = 6, 8
a = encrypt(x)
b = encrypt(y)
c = add(a, b)
print("Shares encrypting x: " + str(a))
print("Shares encrypting y: " + str(b))
print("Sum of shares: " + str(c))
print("Sum of original values (x + y): " + str(decrypt(c)))

## Secret Sharing Using Pysyft

In [ ]:
jake = sy.VirtualWorker(hook, id="jake")
john = sy.VirtualWorker(hook, id="john")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

jake.add_workers([john, secure_worker])
john.add_workers([jake, secure_worker])
secure_worker.add_workers([jake, john])

print("Jake has: " + str(jake._objects))
print("John has: " + str(john._objects))
print("Secure_worker has: " + str(secure_worker._objects))

In [ ]:
x = torch.tensor([6])
x = x.share(jake, john, secure_worker)
print("x: " + str(x))

print("Jake has: " + str(jake._objects))
print("John has: " + str(john._objects))
print("Secure_worker has: " + str(secure_worker._objects))

In [ ]:
y = torch.tensor([8])
y = y.share(jake, john, secure_worker)
print(y)

In [ ]:
z = x + y
print(z)

In [ ]:
z = z.get()
print(z)

# Federated Learning

## Importing Dependencies

In [ ]:
import torch
import torchvision
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms

## Loading Datasets

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, ), (0.5, )),
])

train_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=True, download=True, transform=transform)
test_set = datasets.MNIST(
    "~/.pytorch/MNIST_data/", train=False, download=True, transform=transform)

federated_train_loader = sy.FederatedDataLoader(
    train_set.federate((jake, john)), batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    test_set, batch_size=64, shuffle=True)

## Building the Model

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


model = Model()
optimizer = optim.SGD(model.parameters(), lr=0.01)

## Training the Model

In [ ]:
for epoch in range(0, 5):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):
        # send the model to the client device where the data is present
        model.send(data.location)
        # training the model
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # get back the improved model
        model.get()
        if batch_idx % 100 == 0:
            # get back the loss
            loss = loss.get()
            print('Epoch: {:2d} [{:5d}/{:5d} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                batch_idx * 64,
                len(federated_train_loader) * 64,
                100. * batch_idx / len(federated_train_loader),
                loss.item()))

## Testing the Model

In [ ]:
model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        test_loss += F.nll_loss(
            output, target, reduction='sum').item()
        # get the index of the max log-probability
        pred = output.argmax(1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss,
    correct,
    len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))